In [ ]:
# --target=/home/work/IROLIM numpy

In [1]:
import sys
sys.path.append('/home/work/IROLIM')

In [2]:
!pip install --no-deps --target=/home/work/IROLIM svgpathtools
!pip install --target=/home/work/IROLIM svgwrite

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/bitsandbytes-0.45.4.dev0-py3.12-linux-x86_64.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/looseversion-1.3.0-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/lightning_utilities-0.12.0.dev0-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found

### 불화 파츠 SVG 수집

In [3]:
from svgpathtools import svg2paths, wsvg
import os
import pandas as pd

In [4]:
input_svg_folder = './svg/raw/'
output_parts_folder = './svg/parts/'

os.makedirs(output_parts_folder, exist_ok=True)

for filename in os.listdir(input_svg_folder):
    if filename.endswith('.svg'):
        paths, attributes = svg2paths(os.path.join(input_svg_folder, filename))
        for idx, path in enumerate(paths):
            part_filename = f"{filename[:-4]}_part{idx}.svg"
            with open(os.path.join(output_parts_folder, part_filename), 'w') as f:
                f.write(f'<svg xmlns="http://www.w3.org/2000/svg"><path d="{path.d()}" /></svg>')


### 실루엣만 가져와서 원점조정

In [17]:
def export_svg_part_with_adjusted_viewbox(path_object, output_filepath, padding=10, fill_color="none"):
    """
    단일 SVG path 객체를 새로운 SVG 파일로 저장하고, 해당 path가 viewBox 안에 오도록 좌표를 정렬합니다.

    Args:
        path_object (svgpathtools.Path): 저장할 Path 객체.
        output_filepath (str): 저장할 SVG 파일의 전체 경로.
        padding (int, optional): viewBox에 추가할 여백 (픽셀). 기본값은 10.
        fill_color (str, optional): 저장될 path의 fill 색상. 기본값은 "black".
    """
    if not path_object or path_object.length() == 0:
        print(f"경고: '{output_filepath}'에 저장할 유효한 패스가 없습니다. 건너뜜니다.")
        return

    # 1. path_object의 경계 박스 계산
    p_min_x, p_max_x, p_min_y, p_max_y = path_object.bbox()
    
    # 2. 좌표 변환 값 결정 (path를 (0,0) 근처로 이동)
    translate_x = -p_min_x
    translate_y = -p_min_y

    # 3. path 변환
    translated_path = path_object.translated(complex(translate_x, translate_y))

    # 4. 새로운 viewBox 설정
    # 변환된 패스의 새로운 경계 박스 (0,0 근처에서 시작)
    # 패딩을 고려하여 viewBox의 시작점과 크기를 조정
    vb_width = (p_max_x - p_min_x) + 2 * padding
    vb_height = (p_max_y - p_min_y) + 2 * padding
    
    # viewBox는 "min-x min-y width height" 형식이므로, 패딩을 적용하면 min-x와 min-y를 음수로 시작
    new_viewbox_str = f"{-padding:.2f} {-padding:.2f} {vb_width:.2f} {vb_height:.2f}"

    # 5. 새로운 SVG 파일 생성 및 저장
    # wsvg 함수를 사용하여 path 객체를 SVG 파일로 저장
    # attributes는 <svg> 태그에 적용될 속성들을 담는 딕셔너리의 리스트입니다.
    svg_attrs = {
        'xmlns': "http://www.w3.org/2000/svg",
        'viewBox': new_viewbox_str
    }
    
    # Path 객체의 'fill' 속성 설정 (wsvg가 path 객체의 속성을 우선함)
    # translated_path.fill = fill_color
    
    # wsvg 함수는 단일 path 객체도 리스트 형태로 받습니다.
    wsvg([translated_path], filename=output_filepath, attributes=[svg_attrs])

    # 선택 사항: 콘솔 출력
    # print(f"'{os.path.basename(output_filepath)}' 저장 완료. viewBox: {new_viewbox_str}")
    # 데이터프레임 생성
    data = {
        "X": p_min_x,
        "Y": p_min_y
    }
    return data


In [6]:
from PIL import Image

In [7]:

def get_image_size(filepath, filename):
    """
    이미지 파일의 크기를 반환하는 함수.
    
    Args:
        filepath (str): 이미지 파일의 전체 경로.
        filename (str): 이미지 파일 이름.
    
    Returns:
        dict: 파일명, 너비, 높이를 포함한 딕셔너리. 오류 발생 시 None 반환.
    """
    print(filename)
    try:
        with Image.open(filepath) as img:
            width, height = img.size
            print({"파일명": filename, "너비": width, "높이": height})
    except Exception as e:
        print(f"이미지 처리 실패: {filename}, 오류: {e}")
        return None

In [22]:
# --- 메인 스크립트 (이전 코드에 함수 호출 추가) ---
input_svg_folder = './svg/raw/'
output_parts_folder = './svg/parts/'
os.makedirs(output_parts_folder, exist_ok=True)

print(f"'{input_svg_folder}' 폴더에서 원본 SVG 파일을 스캔하여 파츠별로 분리 및 정렬합니다.")

# 결과를 저장할 리스트
results = []

for filename in os.listdir(input_svg_folder):
    if filename.endswith('.svg'):
        input_filepath = os.path.join(input_svg_folder, filename)
        # get_image_size(input_svg_folder, filename)
        
        try:
            paths, attributes = svg2paths(input_filepath) # 원본 파일의 모든 패스와 속성 읽기
            
            if not paths:
                print(f"경고: '{filename}' 파일에 유효한 패스(path)가 없어 분리할 파츠가 없습니다. 건너뜜니다.")
                continue

            for idx, path in enumerate(paths):
                part_filename = f"{filename[:-4]}_part{idx}.svg"
                part_output_filepath = os.path.join(output_parts_folder, part_filename)
                
                # 새로 정의한 함수를 호출하여 각 파츠를 개별 SVG 파일로 저장하면서 좌표 정렬
                result = export_svg_part_with_adjusted_viewbox(path, part_output_filepath)
                results.append({"X": result["X"], "Y": result["Y"], "Filename": part_filename})
            print(f"'{filename}' 파일이 {len(paths)}개의 파츠로 분리되어 '{output_parts_folder}'에 저장되었습니다.")
        except Exception as e:
            print(f"오류 발생 - '{filename}' 처리 중: {e}")

print("모든 SVG 파일 파츠 분리 및 정렬을 완료했습니다.")

'./svg/raw/' 폴더에서 원본 SVG 파일을 스캔하여 파츠별로 분리 및 정렬합니다.
'1-2.svg' 파일이 4개의 파츠로 분리되어 './svg/parts/'에 저장되었습니다.
모든 SVG 파일 파츠 분리 및 정렬을 완료했습니다.


In [25]:
results

[{'X': 340.9423017075062,
  'Y': 239.61999999999986,
  'Filename': '1-2_part0.svg'},
 {'X': 171.25752676703542, 'Y': 327.881247579185, 'Filename': '1-2_part1.svg'},
 {'X': 338.1047362244047, 'Y': 796.2129611394786, 'Filename': '1-2_part2.svg'},
 {'X': 438.0407715155567, 'Y': 865.0788545542258, 'Filename': '1-2_part3.svg'}]

In [26]:
# 데이터프레임 생성
df = pd.DataFrame(results)
# df["x좌표"] = df["X"].apply(lambda x: round(x, 2))
# df["y좌표"] = df["Y"].apply(lambda y: round(y, 2))


In [27]:
df #원본 캔버스 사이즈에 맞는 파트들의 좌표 시작점.(원점이 됨)

,X,Y,Filename
0,340.942302,239.620000,1-2_part0.svg
1,171.257527,327.881248,1-2_part1.svg
2,338.104736,796.212961,1-2_part2.svg
3,438.040772,865.078855,1-2_part3.svg


### 좌표 고정 부위별 SVG

In [32]:
from svgpathtools import svg2paths2
from bs4 import BeautifulSoup
import os

input_svg_folder = './svg/raw/'
output_parts_folder = './svg/parts/'

os.makedirs(output_parts_folder, exist_ok=True)

for filename in os.listdir(input_svg_folder):
    if filename.endswith('.svg'):
        input_path = os.path.join(input_svg_folder, filename)

        # ---------- Step 1: 원본 SVG에서 이미지 width, height, viewBox 읽기 ----------
        with open(input_path, 'r', encoding='utf-8') as f:
            soup = BeautifulSoup(f, 'xml')
            svg_tag = soup.find('svg')

            # ① 기본 viewBox, width, height 값 가져오기
            viewBox = svg_tag.get('viewBox')
            svg_width = svg_tag.get('width')
            svg_height = svg_tag.get('height')

            # ② <image> 태그에서 원본 이미지 크기 읽기
            image_tag = soup.find('image')
            if image_tag:
                image_width = image_tag.get('width')
                image_height = image_tag.get('height')

                # 만약 SVG width/height 없을 경우 <image> 크기로 설정
                if svg_width is None:
                    svg_width = image_width
                if svg_height is None:
                    svg_height = image_height
            else:
                print(f"경고: {filename} 안에 <image> 태그 없음. width/height 정보 부족할 수 있음.")

        # ---------- Step 2: Path 추출 ----------
        paths, attributes, svg_attributes = svg2paths2(input_path)

        for idx, path in enumerate(paths):
            part_filename = f"{filename[:-4]}_part{idx}.svg"
            output_path = os.path.join(output_parts_folder, part_filename)

            attr = attributes[idx]
            fill = attr.get('fill', 'none')
            stroke = attr.get('stroke', 'none')
            style = attr.get('style', '')

            # ---------- Step 3: 파츠별 SVG 저장 (원본 좌표계 + viewBox 유지) ----------
            with open(output_path, 'w', encoding='utf-8') as f:
                f.write(f'<svg xmlns="http://www.w3.org/2000/svg"')

                # viewBox 설정
                if viewBox:
                    f.write(f' viewBox="{viewBox}"')

                # width/height 설정
                if svg_width:
                    f.write(f' width="{svg_width}"')
                if svg_height:
                    f.write(f' height="{svg_height}"')
                f.write('>\n')

                # path 저장 (좌표 수정 없이 그대로 원본 절대좌표)
                style_attr = f' style="{style}"' if style else ''
                f.write(f'<path d="{path.d()}" fill="{fill}" stroke="{stroke}"{style_attr} />\n')
                f.write('</svg>\n')

        print(f"{filename} → {len(paths)}개의 파츠로 분해 완료 (viewBox={viewBox}, width={svg_width}, height={svg_height})")


1-2.svg → 4개의 파츠로 분해 완료 (viewBox=0 0 900.24 1200, width=3751, height=5000)
